In [4]:
import os, sys
import sqlite3, csv, glob

In [6]:
SEP =','
def define_field(s):
    try:
        int(s)
        return 'integer'
    except ValueError:
        try:
            float(s)
            return 'real'
        except:
            return 'text'
        

In [8]:
def create_sqlite_db(db='database.sqlite', file_pattern=''):
    conn = sqlite3.connect(db)
    conn.text_factory = str # utf-8
    c = conn.cursor()
    
    target_file = glob.glob(file_pattern)
    print('Creating %i table(s) into %s from file(s): %s' % (len(target_files), db, ''.join(target_files)))
    
    for k, csvfile in enumerator(target_files):
        tablename = os.path.splitext(os.path.basename(csvfile))[0]
        with open(csvfile, 'r') as f:
            reader = csv.reader(f,delimiter=SEP)
            f.seek(0)
            for n, row in enumerate(reader):
                if n==11:
                    types = map(define_field,row)
                else:
                    if n>11:
                        break
            
            f.seek(0)
            for n, row in enumerate(reader):
                if n==0:
                    sql = "DROP TABLE IF EXISTS %s" % tablename
                    c.execute(sql)
                    sql = "CREATE TABLE %s (%s)" % (tablename, ",".join(["%s %s" % (col, ct) for col, ct in zip(row, types)]))
                    print('%i) %s' % (k+1, sql))
                    c.execute(sql)
                    
                    # indexing
                    for column in row:
                        if column.endswith("_ID_hash"):
                            index = "%s__%s" %(tablename,column)
                            sql = "CREATE INDEX %s on %s (%s)" % (index,tablename,column)
                            c.excute(sql)
                    
                    insertsql = "INSERT INTO %s VALUES (%s)" %(tablename, ",".join(["?" for column in row]))
                    rowlen = len(row)
                else:
                    if len(row) == rowlen:
                        c.excute(insertsql, row)
                    else:
                        print('Error at line %i in file %s') %(n,csvfile) 
                        raise ValueError('Houston, we\'ve had a problem at row %i' %n)
        conn.commit()
        print('* Inserted %i rows' %n)
    c.close()
    conn.close()
    